In [ ]:
#  Implement Logistic regression With Neural Network Mindset.
#  ● logistic regression classifier for classification. 
#  ● Plot the loss over each epoch.
#  ● Plot the accuracy over each epoch
# ● Report final Accuracy

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

# Derivative of activation functions
def sigmoid_derivative(a):
    return a * (1 - a)

def relu_derivative(a):
    return (a > 0).astype(float)

# Loss functions
def binary_cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    return -1/m * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def mean_squared_error(y_true, y_pred):
    m = y_true.shape[0]
    return np.sum((y_true - y_pred) ** 2) / m

# Derivative of loss functions
def binary_cross_entropy_derivative(y_true, y_pred):
    return y_pred - y_true

def mean_squared_error_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

# Prompt user to select activation and loss functions
print("Select activation function:")
print("1. Sigmoid")
print("2. ReLU")
activation_choice = input("Enter choice (1/2): ")

print("Select loss function:")
print("1. Binary Cross-Entropy")
print("2. Mean Squared Error")
loss_choice = input("Enter choice (1/2): ")

# Set activation and loss functions based on user choice
if activation_choice == "1":
    activation_fn = sigmoid
    activation_derivative_fn = sigmoid_derivative
elif activation_choice == "2":
    activation_fn = relu
    activation_derivative_fn = relu_derivative
else:
    raise ValueError("Invalid activation function choice")

if loss_choice == "1":
    loss_fn = binary_cross_entropy
    loss_derivative_fn = binary_cross_entropy_derivative
elif loss_choice == "2":
    loss_fn = mean_squared_error
    loss_derivative_fn = mean_squared_error_derivative
else:
    raise ValueError("Invalid loss function choice")

# Generate a binary classification dataset
X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, random_state=1)
y = y.reshape(y.shape[0], 1)  # Reshape y to be a column vector

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Function to train the logistic regression model
def train(X, y, learning_rate=0.01, epochs=1000):
    m, n = X.shape
    W = np.zeros((n, 1))
    b = 0
    losses = []
    accuracies = []

    for epoch in range(epochs):
        # Forward pass
        Z = np.dot(X, W) + b
        A = activation_fn(Z)

        # Compute loss and accuracy
        loss = loss_fn(y, A)
        losses.append(loss)
        predictions = A > 0.5
        acc = accuracy_score(y, predictions)
        accuracies.append(acc)

        # Backward pass (gradient descent)
        dZ = loss_derivative_fn(y, A) * activation_derivative_fn(A)
        dW = (1/m) * np.dot(X.T, dZ)
        db = (1/m) * np.sum(dZ)

        # Update weights and bias
        W -= learning_rate * dW
        b -= learning_rate * db

    return W, b, losses, accuracies

# Train the model and get the loss and accuracy over epochs
W, b, losses, accuracies = train(X_train, y_train, learning_rate=0.01, epochs=1000)

# Plot loss and accuracy over epochs
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(losses, label="Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracies, label="Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy over Epochs")
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test set
Z_test = np.dot(X_test, W) + b
A_test = activation_fn(Z_test)
y_pred_test = A_test > 0.5
final_accuracy = accuracy_score(y_test, y_pred_test)

print("Final Test Accuracy:", final_accuracy)


In [ ]:
#  Implement Shallow Neural Network model:
#  ● Implement a binary classifi cation neural network with a single and multiple hidden layers.
#  ● Implement a Multi-class classifi cation neural network with a single and multiple hidden 
# layers.
#  ● Vary the number of neurons at suitable layers

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Activation functions and their derivatives
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def tanh(z):
    return np.tanh(z)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def sigmoid_derivative(a):
    return a * (1 - a)

def relu_derivative(a):
    return np.where(a > 0, 1, 0)

def tanh_derivative(a):
    return 1 - np.square(a)

# Loss functions
def binary_cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    return -1/m * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def categorical_cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    return -1/m * np.sum(y_true * np.log(y_pred))

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Neural network model class
class ShallowNeuralNetwork:
    def __init__(self, layer_dims, output_activation='sigmoid', hidden_activation='relu', loss='binary_cross_entropy'):
        self.layer_dims = layer_dims
        self.output_activation = output_activation
        self.hidden_activation = hidden_activation
        self.loss = loss
        self.params = self.initialize_params()

    # Initialize weights and biases
    def initialize_params(self):
        np.random.seed(1)
        params = {}
        for l in range(1, len(self.layer_dims)):
            params[f"W{l}"] = np.random.randn(self.layer_dims[l], self.layer_dims[l-1]) * 0.01
            params[f"b{l}"] = np.zeros((self.layer_dims[l], 1))
        return params

    # Forward pass
    def forward_propagation(self, X):
        A = X
        cache = {"A0": A}
        L = len(self.layer_dims) - 1

        # Hidden layers
        for l in range(1, L):
            Z = np.dot(self.params[f"W{l}"], A) + self.params[f"b{l}"]
            if self.hidden_activation == 'sigmoid':
                A = sigmoid(Z)
            elif self.hidden_activation == 'relu':
                A = relu(Z)
            elif self.hidden_activation == 'tanh':
                A = tanh(Z)
            cache[f"A{l}"], cache[f"Z{l}"] = A, Z

        # Output layer
        ZL = np.dot(self.params[f"W{L}"], A) + self.params[f"b{L}"]
        if self.output_activation == "sigmoid":
            AL = sigmoid(ZL)
        elif self.output_activation == "softmax":
            AL = softmax(ZL)
        elif self.output_activation == "relu":
            AL = relu(ZL)
        cache[f"A{L}"], cache[f"Z{L}"] = AL, ZL
        return AL, cache

    # Backward pass
    def backward_propagation(self, X, y, cache):
        m = X.shape[1]
        grads = {}
        L = len(self.layer_dims) - 1
        AL = cache[f"A{L}"]

        # Loss function derivatives
        if self.loss == 'binary_cross_entropy':
            dZL = AL - y
        elif self.loss == 'categorical_cross_entropy':
            dZL = AL - y
        elif self.loss == 'mean_squared_error':
            dZL = AL - y

        # Output layer gradients
        grads[f"dW{L}"] = (1/m) * np.dot(dZL, cache[f"A{L-1}"].T)
        grads[f"db{L}"] = (1/m) * np.sum(dZL, axis=1, keepdims=True)

        # Hidden layers gradients
        for l in reversed(range(1, L)):
            dA = np.dot(self.params[f"W{l+1}"].T, dZL)
            if self.hidden_activation == 'sigmoid':
                dZ = dA * sigmoid_derivative(cache[f"A{l}"])
            elif self.hidden_activation == 'relu':
                dZ = dA * relu_derivative(cache[f"A{l}"])
            elif self.hidden_activation == 'tanh':
                dZ = dA * tanh_derivative(cache[f"A{l}"])
            grads[f"dW{l}"] = (1/m) * np.dot(dZ, cache[f"A{l-1}"].T)
            grads[f"db{l}"] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            dZL = dZ

        return grads

    # Update parameters
    def update_params(self, grads, learning_rate):
        for l in range(1, len(self.layer_dims)):
            self.params[f"W{l}"] -= learning_rate * grads[f"dW{l}"]
            self.params[f"b{l}"] -= learning_rate * grads[f"db{l}"]

    # Train model
    def train(self, X, y, epochs, learning_rate):
        losses = []
        for epoch in range(epochs):
            AL, cache = self.forward_propagation(X)
            if self.loss == "binary_cross_entropy":
                loss = binary_cross_entropy(y, AL)
            elif self.loss == "categorical_cross_entropy":
                loss = categorical_cross_entropy(y, AL)
            elif self.loss == "mean_squared_error":
                loss = mean_squared_error(y, AL)
            losses.append(loss)
            grads = self.backward_propagation(X, y, cache)
            self.update_params(grads, learning_rate)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

        return losses

    # Predict
    def predict(self, X):
        AL, _ = self.forward_propagation(X)
        if self.output_activation == "sigmoid":
            return AL > 0.5
        elif self.output_activation == "softmax":
            return np.argmax(AL, axis=0)

# Prepare dataset
def prepare_data(binary=True):
    if binary:
        X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, random_state=1)
        y = y.reshape(y.shape[0], 1)
        output_activation = "sigmoid"
        loss = "binary_cross_entropy"
    else:
        X, y = make_blobs(n_samples=1000, centers=3, n_features=2, random_state=1)
        y = y.reshape(y.shape[0], 1)
        enc = OneHotEncoder(sparse=False)
        y = enc.fit_transform(y)
        output_activation = "softmax"
        loss = "categorical_cross_entropy"
    return X, y, output_activation, loss

# Binary Classification Example
X, y, output_activation, loss = prepare_data(binary=True)
X_train, X_test, y_train, y_test = train_test_split(X.T, y.T, test_size=0.2, random_state=1)
layer_dims = [X_train.shape[0], 5, 1]
model = ShallowNeuralNetwork(layer_dims, output_activation=output_activation, hidden_activation='relu', loss=loss)
losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.01)

# Plot Loss
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.show()

# Predict and Accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test.T, y_pred.T)
print(f"Final Test Accuracy: {accuracy}")


In [ ]:
#  Hyper parameter Tuning of a Neural Network model implemented for hand-written digit 
# classifi cation:
#  ● Vary the type of activation functions.
#  ● Choose suitable Loss functions.
#  ● Vary the number of neurons at suitable layers.
#  ● Vary Weight Initialization methods.
#  ● Save the Best Model and load the saved model

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import ParameterGrid
import numpy as np

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define hyperparameter grid
param_grid = {
    'activation': ['relu', 'sigmoid', 'tanh'],
    'loss': ['categorical_crossentropy', 'mean_squared_error'],
    'neurons': [64, 128],
    'initializer': ['he_uniform', 'glorot_uniform', 'random_normal']
}

# Placeholder for best model and its score
best_model = None
best_accuracy = 0
best_params = {}

# Hyperparameter tuning loop
for params in ParameterGrid(param_grid):
    print(f"Testing with parameters: {params}")
    
    # Define the model
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(params['neurons'], activation=params['activation'], kernel_initializer=params['initializer']),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss=params['loss'], metrics=['accuracy'])
    
    # Define callbacks: save the model if it has the best accuracy
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint, early_stop], verbose=1)
    
    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Validation Accuracy: {val_accuracy}")
    
    # Update best model
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model
        best_params = params

print(f"\nBest Model Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_accuracy}")

# Load the best saved model
best_model = tf.keras.models.load_model('best_model.h5')
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy of Best Model: {test_accuracy}")


In [ ]:
# Building a Deep Neural Network:  
# ● Implement a multi-class classifi cation neural network with number of layers of your choice.
#  ● Include Batch Normalization layers.
#  ● Vary Optimization methods.
#  ● Add drop out layers.

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import ParameterGrid
import numpy as np

# Load and preprocess MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define a grid of hyperparameters to vary optimizers and dropout rates
param_grid = {
    'optimizer': [Adam(), SGD(), RMSprop()],
    'dropout_rate': [0.2, 0.4],
    'batch_norm': [True, False]
}

best_model = None
best_accuracy = 0
best_params = {}

# Loop over hyperparameter combinations
for params in ParameterGrid(param_grid):
    print(f"\nTesting with parameters: {params}")
    
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    
    # Add layers with batch normalization and dropout
    for _ in range(3):  # Using 3 hidden layers as an example
        model.add(Dense(128, activation='relu'))
        if params['batch_norm']:
            model.add(BatchNormalization())
        model.add(Dropout(params['dropout_rate']))
    
    model.add(Dense(10, activation='softmax'))  # Output layer
    
    # Compile model with chosen optimizer
    model.compile(optimizer=params['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Callbacks for saving the best model
    checkpoint = ModelCheckpoint('best_deep_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint, early_stop], verbose=1)
    
    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Validation Accuracy: {val_accuracy}")
    
    # Track the best model
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model
        best_params = params

# Output the best parameters and accuracy
print(f"\nBest Model Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_accuracy}")

# Load and evaluate the best saved model
best_model = tf.keras.models.load_model('best_deep_model.h5')
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy of Best Model: {test_accuracy}")


In [ ]:
#  Convolutional Neural Network Models.
#  ● Design a Convolutional neural network with the layers of your choice
#  ● Compare the performance by changing the 
# ● Kernel size
#  ● Number of feature maps at each convolutional layer
#  ● Stride.
#  ● Padding.
#  ● Number of fully connected layers.

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterGrid

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0  # Reshape and normalize
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define hyperparameter grid
param_grid = {
    'kernel_size': [(3, 3), (5, 5)],
    'feature_maps': [32, 64],
    'stride': [1, 2],
    'padding': ['same', 'valid'],
    'fully_connected_layers': [1, 2]
}

best_model = None
best_accuracy = 0
best_params = {}

# Hyperparameter tuning loop
for params in ParameterGrid(param_grid):
    print(f"\nTesting with parameters: {params}")
    
    model = Sequential()
    
    # First convolutional layer
    model.add(Conv2D(filters=params['feature_maps'],
                     kernel_size=params['kernel_size'],
                     strides=params['stride'],
                     padding=params['padding'],
                     activation='relu',
                     input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second convolutional layer with doubled filters
    model.add(Conv2D(filters=params['feature_maps'] * 2,
                     kernel_size=params['kernel_size'],
                     strides=params['stride'],
                     padding=params['padding'],
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    # Add fully connected layers
    for _ in range(params['fully_connected_layers']):
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define checkpoint to save the best model
    checkpoint = ModelCheckpoint('best_cnn_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=32, callbacks=[checkpoint], verbose=1)
    
    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Validation Accuracy: {val_accuracy}")
    
    # Track the best model
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model
        best_params = params

# Output the best parameters and accuracy
print(f"\nBest Model Parameters: {best_params}")
print(f"Best Validation Accuracy: {best_accuracy}")

# Load and evaluate the best saved model
best_model = tf.keras.models.load_model('best_cnn_model.h5')
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy of Best Model: {test_accuracy}")


In [ ]:
# Visualization of CNN Models.
#  ● Design a Convolutional Neural Network Model for image classifi cation.
#  ● Plot Model Architecture.
#  ● Visualize feature maps after training of CNN.
#  ● Visualize class activation maps

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0  # Reshape and normalize
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Step 1: Design and compile a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=32)

# Plot the model architecture
tf.keras.utils.plot_model(model, to_file="model_architecture.png", show_shapes=True)

# Step 2: Visualize Feature Maps for a Sample Image
def plot_feature_maps(model, layer_indices, input_image):
    for layer_index in layer_indices:
        layer_model = Model(inputs=model.input, outputs=model.layers[layer_index].output)
        feature_maps = layer_model.predict(input_image)
        
        # Plot each feature map
        num_feature_maps = feature_maps.shape[-1]
        fig, axes = plt.subplots(1, num_feature_maps, figsize=(15, 15))
        for i in range(num_feature_maps):
            axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
            axes[i].axis('off')
        plt.show()

# Select a sample image
sample_image = X_test[0].reshape(1, 28, 28, 1)

# Visualize feature maps for the first and second convolutional layers
plot_feature_maps(model, layer_indices=[0, 2], input_image=sample_image)

# Step 3: Visualize Class Activation Maps (CAMs)
def generate_cam(model, input_image, class_index):
    # Create a model that outputs the feature maps from the last conv layer
    last_conv_layer = model.get_layer("conv2d_1")
    grad_model = Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_image)
        loss = predictions[:, class_index]

    # Compute gradients of the target class wrt the last conv layer output
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight the conv layer outputs by the pooled gradients
    conv_outputs = conv_outputs[0]
    cam = np.zeros(conv_outputs.shape[:2], dtype=np.float32)
    for i, w in enumerate(pooled_grads):
        cam += w * conv_outputs[:, :, i]
    
    # Apply ReLU and normalize CAM
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)

    # Resize to match input image dimensions and plot
    cam = tf.image.resize(cam[..., tf.newaxis], (28, 28)).numpy()
    plt.imshow(input_image[0].reshape(28, 28), cmap='gray')
    plt.imshow(cam, cmap='jet', alpha=0.5)  # Overlay CAM with original image
    plt.title("Class Activation Map (CAM)")
    plt.axis('off')
    plt.show()

# Generate CAM for a specific class (e.g., the predicted class of the sample image)
predicted_class = np.argmax(model.predict(sample_image))
generate_cam(model, sample_image, predicted_class)


In [ ]:
# Using Deep pre-trained CNN model for feature extraction:
#  ● Extract features from the FC1 of VGG network.
#  ● Train any traditional ML model like SVM for classifi cation. 
# ● Repeat the above by considering FC2 of VGG for feature extraction.

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.layers import Flatten

# Step 1: Load VGG16 or VGG19 pre-trained model without the top layer (to remove FC layers)
vgg_model = VGG16(weights='imagenet', include_top=True)
# If using VGG19:
# vgg_model = VGG19(weights='imagenet', include_top=True)

# Step 2: Create new models to extract features from FC1 and FC2 layers
# FC1 Layer - Flatten layer after the fully connected 1 layer
fc1_model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('fc1').output)

# FC2 Layer - Flatten layer after the fully connected 2 layer
fc2_model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('fc2').output)

# Function to preprocess an image and extract features from the specified layer
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))  # VGG input size: 224x224
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)  # Preprocessing for VGG
    features = model.predict(img_array)
    return features.flatten()

# Step 3: Load the dataset
# Let's assume you have a folder with images for each class (e.g., 'class1', 'class2')
# For simplicity, assuming folder structure:
# - data/
#   - class1/
#   - class2/
#   - ...
dataset_path = '/path/to/dataset/'
class_names = os.listdir(dataset_path)

# Prepare lists to store features and labels
features_fc1 = []
features_fc2 = []
labels = []

# Loop through all the classes and extract features
for label, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        
        # Extract features from FC1 and FC2 layers
        fc1_features = extract_features(img_path, fc1_model)
        fc2_features = extract_features(img_path, fc2_model)
        
        features_fc1.append(fc1_features)
        features_fc2.append(fc2_features)
        labels.append(label)

# Step 4: Convert lists to numpy arrays
features_fc1 = np.array(features_fc1)
features_fc2 = np.array(features_fc2)
labels = np.array(labels)

# Step 5: Scale the features
scaler = StandardScaler()
features_fc1_scaled = scaler.fit_transform(features_fc1)
features_fc2_scaled = scaler.fit_transform(features_fc2)

# Step 6: Split the data into training and testing sets
X_train_fc1, X_test_fc1, y_train, y_test = train_test_split(features_fc1_scaled, labels, test_size=0.2, random_state=42)
X_train_fc2, X_test_fc2 = train_test_split(features_fc2_scaled, test_size=0.2, random_state=42)

# Step 7: Train an SVM classifier on the features extracted from FC1
svm_fc1 = SVC(kernel='linear')  # You can experiment with different SVM kernels
svm_fc1.fit(X_train_fc1, y_train)

# Step 8: Evaluate the SVM model for FC1
y_pred_fc1 = svm_fc1.predict(X_test_fc1)
print("Classification report for FC1 features:")
print(classification_report(y_test, y_pred_fc1))

# Step 9: Train an SVM classifier on the features extracted from FC2
svm_fc2 = SVC(kernel='linear')  # You can experiment with different SVM kernels
svm_fc2.fit(X_train_fc2, y_train)

# Step 10: Evaluate the SVM model for FC2
y_pred_fc2 = svm_fc2.predict(X_test_fc2)
print("Classification report for FC2 features:")
print(classification_report(y_test, y_pred_fc2))


In [ ]:
#  Fine-tuning Deep pre-trained CNN for Classifi cation:
#  ● Fine-tune VGG network for the task under consideration.
#  ● Check the performance by making.
#  ● all the layers trainable.
#  ● freezing the initial layers.
#  ● freezing the entire network except the fi nal layer.

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Function to load selected pre-trained model
def load_pretrained_model(model_name, input_shape, num_classes):
    if model_name == "VGG16":
        base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == "VGG19":
        base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == "ResNet50":
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == "AlexNet":
        base_model = create_alexnet(input_shape)
    else:
        print("Invalid model name selected.")
        return None
    
    # Add custom top layers for classification
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    return model

# Function to create AlexNet
def create_alexnet(input_shape):
    model = tf.keras.Sequential([
        Conv2D(96, (11, 11), strides=4, padding='valid', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(256, (5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1000, activation='softmax')
    ])
    return model

# Display available models and prompt user to select one
print("Available Models: [VGG16, VGG19, ResNet50, AlexNet]")
selected_model = input("Please enter the model you want to use: ")

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, rotation_range=15)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    'path_to_train_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'path_to_validation_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the selected model
input_shape = (224, 224, 3)
num_classes = train_generator.num_classes
model = load_pretrained_model(selected_model, input_shape, num_classes)

if model:
    # Helper function to compile and train the model
    def compile_and_train(model, trainable_layers, epochs=5):
        for layer in model.layers:
            layer.trainable = layer.name in trainable_layers
        
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
        history = model.fit(
            train_generator,
            epochs=epochs,
            validation_data=val_generator
        )
        return history

    # Fine-tuning strategy examples
    print("Training all layers...")
    all_layers = [layer.name for layer in model.layers]
    history_all_trainable = compile_and_train(model, all_layers)

    print("Training with frozen initial layers...")
    initial_frozen_layers = [layer.name for layer in model.layers[10:]]
    history_initial_frozen = compile_and_train(model, initial_frozen_layers)

    print("Training with only the final layer trainable...")
    final_layer_only = [model.layers[-1].name]
    history_final_layer_only = compile_and_train(model, final_layer_only)

    # Plotting performance for each strategy
    def plot_history(history, label):
        plt.plot(history.history['accuracy'], label=f'{label} - Train Accuracy')
        plt.plot(history.history['val_accuracy'], label=f'{label} - Val Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    plot_history(history_all_trainable, "All Layers Trainable")
    plot_history(history_initial_frozen, "Initial Layers Frozen")
    plot_history(history_final_layer_only, "Final Layer Only")
else:
    print("No valid model selected.")


In [ ]:
#  Design MLFFNN with 3-level stacked autoencoder based pre-training for Black and white 
# image data, Display features extracted by diff erent levels of stacked autoencoder at the end 
# of pre-training.

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset (black and white images)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the autoencoder structure
def create_autoencoder(input_shape, encoding_dim):
    input_img = Input(shape=input_shape)
    x = Flatten()(input_img)
    x = Dense(encoding_dim, activation='relu')(x)
    encoded = Dense(encoding_dim, activation='relu')(x)
    
    # Decoder
    x = Dense(encoding_dim, activation='relu')(encoded)
    x = Dense(np.prod(input_shape), activation='sigmoid')(x)
    decoded = Reshape(input_shape)(x)
    
    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)
    
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Create 3 levels of stacked autoencoders
input_shape = (28, 28, 1)
encoding_dim1 = 128
encoding_dim2 = 64
encoding_dim3 = 32

# Level 1 Autoencoder
autoencoder1, encoder1 = create_autoencoder(input_shape, encoding_dim1)

# Train first autoencoder
autoencoder1.fit(x_train, x_train, epochs=5, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Extract features from the first autoencoder (encoded features)
x_train_encoded1 = encoder1.predict(x_train)
x_test_encoded1 = encoder1.predict(x_test)

# Level 2 Autoencoder
autoencoder2, encoder2 = create_autoencoder((encoding_dim1,), encoding_dim2)

# Train second autoencoder
autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=5, batch_size=256, shuffle=True, validation_data=(x_test_encoded1, x_test_encoded1))

# Extract features from the second autoencoder
x_train_encoded2 = encoder2.predict(x_train_encoded1)
x_test_encoded2 = encoder2.predict(x_test_encoded1)

# Level 3 Autoencoder
autoencoder3, encoder3 = create_autoencoder((encoding_dim2,), encoding_dim3)

# Train third autoencoder
autoencoder3.fit(x_train_encoded2, x_train_encoded2, epochs=5, batch_size=256, shuffle=True, validation_data=(x_test_encoded2, x_test_encoded2))

# Extract features from the third autoencoder
x_train_encoded3 = encoder3.predict(x_train_encoded2)
x_test_encoded3 = encoder3.predict(x_test_encoded2)

# Visualize the extracted features at each level of the stacked autoencoder
def plot_features(features, title):
    plt.figure(figsize=(10, 5))
    plt.imshow(features[0].reshape(1, -1), cmap='gray', aspect='auto')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Plot features at different levels
plot_features(x_train_encoded1, "Features extracted by first autoencoder")
plot_features(x_train_encoded2, "Features extracted by second autoencoder")
plot_features(x_train_encoded3, "Features extracted by third autoencoder")

# Building the MLFFNN for classification
def build_mlffnn(input_dim, num_classes):
    model = tf.keras.Sequential([
        Dense(512, activation='relu', input_dim=input_dim),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Use the features extracted from the third autoencoder as input to the MLFFNN
input_dim = x_train_encoded3.shape[1]
mlffnn = build_mlffnn(input_dim, 10)  # 10 classes for MNIST

# Fine-tune the MLFFNN on the features from the 3rd level of the stacked autoencoder
mlffnn.fit(x_train_encoded3, y_train, epochs=10, batch_size=256, validation_data=(x_test_encoded3, y_test))

# Evaluate the performance of the model
score = mlffnn.evaluate(x_test_encoded3, y_test)
print(f"Test Loss: {score[0]}, Test Accuracy: {score[1]}")


In [ ]:
#  Sentiment Analysis
#  ● Pre-process the text.
#  ● Convert the text into word embeddings.
#  ● Implement the classifi cation network using LSTMs/ GRUs.
#  ● Pre-process the text.
#  ● Convert the text into word embeddings.
#  ● Implement the classifi cation network using LSTMs/ GRUs

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

max_len = 200
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
X_test = pad_sequences(X_test, padding='post', maxlen=max_len)

embedding_dim = 100

def create_model(embedding_dim, max_len, use_gru=False):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(0.2))
    
    if use_gru:
        model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
    else:
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

model = create_model(embedding_dim, max_len, use_gru=False)

history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)

score, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {score}, Test Accuracy: {accuracy}")

import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
